In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.append('/Users/ajc/CODES/ats/tools/utils/')
import get_transect_data

In [3]:
Data_B = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_B['temperature'] = np.array(Data)
Data_B['time'] = np.array(Time)

In [4]:
Data_S30 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S30_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S30['temperature'] = np.array(Data)
Data_S30['time'] = np.array(Time)

In [5]:
Data_S60 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S60_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S60['temperature'] = np.array(Data)
Data_S60['time'] = np.array(Time)

In [6]:
Data_S90 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S90['temperature'] = np.array(Data)
Data_S90['time'] = np.array(Time)

In [7]:
def get_AveMaxMin_Temp(season = " ", start=0, end=-1,depth=25):
    #25 * 2cm = 50 cm
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    
    AvgTemp = np.zeros(cycles)
    MaxTemp = np.zeros(cycles)
    MinTemp = np.zeros(cycles)
    #print depth
    depth_cm = data[1,0,0,80] - data[1,0,0,80-depth]
    for i in range(cycles):
        AvgTemp[i] = np.mean(data[nvar-1,i,start:end,80-depth:])
        MaxTemp[i] = np.max(data[nvar-1,i,start:end,80-depth:])
        MinTemp[i] = np.min(data[nvar-1,i,start:end,80-depth:])
    L = len(AvgTemp)%365
    HillslopeT['times'] = np.array(times[:-L])
    HillslopeT_Avg = np.array(AvgTemp[:-L])
    HillslopeT_Max = np.array(MaxTemp[:-L])
    HillslopeT_Min = np.array(MinTemp[:-L])
   

    HillslopeT['temp-annual_avg'] = np.reshape(HillslopeT_Avg, (-1,365)).mean(axis=1)
    HillslopeT['temp-annual_max'] = np.reshape(HillslopeT_Max, (-1,365)).max(axis=1)
    HillslopeT['temp-annual_min'] = np.reshape(HillslopeT_Min, (-1,365)).min(axis=1)

    return HillslopeT, depth_cm

In [61]:
def get_Temp_timeseries_location(season = " ", location=0,depth=25):
    #25 * 2cm = 50 cm
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT1 = dict()
    
    Temp = np.zeros(cycles)
    
    #print depth
    depth_cm = data[1,0,0,80] - data[1,0,0,80-depth]
    for i in range(cycles):
        Temp[i] = data[nvar-1,i,location,80-depth]

    HillslopeT1['times'] = np.array(times[-365:])
    HillslopeT_Avg = np.array(Temp[-365:])

    HillslopeT1['temp'] = HillslopeT_Avg
    
    return HillslopeT1, depth_cm

In [ ]:
def get_locationbased_ALT(season = " ",location=0):
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    avgTemp = np.zeros(cycles)
    
    ddepth = 0 
    for i in range(cycles):
        loc = np.where(data[nvar-1,i,location,:] > 273.15)[0]
        if len(loc) >0:
            dz = -(80 - loc[0])*2.0
            avgTemp[i] = dz
    HillslopeT['times'] = np.array(times)
    HillslopeT['temp'] = np.array(avgTemp)
    print nvar,len(loc), loc 
    print data[1,0,location,:], data[1,0,location,loc[0]], loc[0]
    L = len(HillslopeT['temp'])%365
    for key in HillslopeT.keys():
        HillslopeT[key + '-annual_avg'] = np.reshape(HillslopeT[key][:-L], (-1,365))   
    return HillslopeT

In [9]:
RiparianTemp = dict()
start1_,end1_,depth_ = 0,5,55 # Z depth 3.84718
#start1_,end1_,depth_ = 0,5,46 # Z depth 2.053

RiparianTempB, Depth_cm = get_AveMaxMin_Temp("B",start=start1_,end=end1_,depth=depth_)
RiparianTempS30,Depth_cm = get_AveMaxMin_Temp("S30",start=start1_,end=end1_,depth=depth_)
RiparianTempS60,Depth_cm = get_AveMaxMin_Temp("S60",start=start1_,end=end1_,depth=depth_)
RiparianTempS90,Depth_cm = get_AveMaxMin_Temp("S90",start=start1_,end=end1_,depth=depth_)
print (Depth_cm)

3.84718


In [10]:
depth_cm =round(100*Depth_cm)

In [59]:
HillslopeTemp = dict()
start2_,end2_ = end1_,99
HillslopeTempB,Depth_cm = get_AveMaxMin_Temp("B",start=start2_,end=end2_,depth=depth_)
HillslopeTempS30,Depth_cm = get_AveMaxMin_Temp("S30",start=start2_,end=end2_,depth=depth_)
HillslopeTempS60,Depth_cm = get_AveMaxMin_Temp("S60",start=start2_,end=end2_,depth=depth_)
HillslopeTempS90,Depth_cm = get_AveMaxMin_Temp("S90",start=start2_,end=end2_,depth=depth_)

In [63]:
depth1=26
WetlandTempB, Depth_cm1 = get_Temp_timeseries_location("B",location=0,depth=depth1)
WetlandTempS30,Depth_cm1 = get_Temp_timeseries_location("S30",location=0,depth=depth1)
WetlandTempS60,Depth_cm1 = get_Temp_timeseries_location("S60",location=0,depth=depth1)
WetlandTempS90,Depth_cm1 = get_Temp_timeseries_location("S90",location=0,depth=depth1)
print Depth_cm1

0.5623399999999998


In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAverage():
    fig, axs = plt.subplots(1,2, figsize=(8,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
    #plt.subplot(121)
    ax1=plt.subplot(121)
    ax1.plot(RiparianTempB['times-annual_avg'], RiparianTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_avg'], RiparianTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_avg'], RiparianTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_avg'], RiparianTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')

    ax1.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((266.5, 271.5))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    ax1.legend(loc='lower left',fontsize=8)
    ax1.set_title('Wetland',fontweight='normal')
    plt.tight_layout()

    ax2=plt.subplot(122)
    ax2.plot(HillslopeTempB['times-annual_avg'], HillslopeTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_avg'], HillslopeTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_avg'], HillslopeTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_avg'], HillslopeTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')


    ax2.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((266.5, 271.5))
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    ax2.legend(loc='upper left',fontsize=8)
    ax2.set_title('Upland',fontweight='normal')

    plt.tight_layout()
    #print 'DONE'
    plt.savefig(outfile+'temp-%scm.pdf'%depth_cm, bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMin():
    fig, axs = plt.subplots(3,2, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
    #plt.subplot(121)
    ax1=plt.subplot(321)
    ax1.plot(RiparianTempB['times-annual_avg'], RiparianTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_avg'], RiparianTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_avg'], RiparianTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_avg'], RiparianTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')

    #ax1.set_xlabel('Year',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((266.5, 271.5))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax1.legend(loc='upper left',fontsize=8)
    ax1.set_title('Wetland',fontweight='normal')
    ax1.text(2, 267, 'Annual average temperature',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(322)
    ax2.plot(HillslopeTempB['times-annual_avg'], HillslopeTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_avg'], HillslopeTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_avg'], HillslopeTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_avg'], HillslopeTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')


    #ax2.set_xlabel('Year',fontsize='large', fontweight='normal')
    #ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((266.5, 271.5))
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax2.legend(loc='upper left',fontsize=8)
    ax2.set_title('Upland',fontweight='normal')
    ax2.text(2, 270.5, 'Annual average temperature',fontsize=11, fontweight='normal')
    
    # --------------------------------- MAXIMUM ---------------------------
    ax1=plt.subplot(323)
    ax1.plot(RiparianTempB['times-annual_max'], RiparianTempB['temp-annual_max'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_max'], RiparianTempS30['temp-annual_max'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_max'], RiparianTempS60['temp-annual_max'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_max'], RiparianTempS90['temp-annual_max'],'m.-',label=r'S$_{90}$')

    #ax1.set_xlabel('Year',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((275.5, 276.2))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax1.set_yticklabels(np.linspace(275.6,276.2,4))
    #ax1.legend(loc='upper left',fontsize=8)
    ax1.text(2, 275.6, 'Annual maximum temperature',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(324)
    ax2.plot(HillslopeTempB['times-annual_max'], HillslopeTempB['temp-annual_max'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_max'], HillslopeTempS30['temp-annual_max'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_max'], HillslopeTempS60['temp-annual_max'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_max'], HillslopeTempS90['temp-annual_max'],'m.-',label=r'S$_{90}$')

    #ax2.set_xlabel('Year',fontsize='large', fontweight='normal')
    #ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((275.5, 276.2))
    ax2.text(2, 275.6, 'Annual maximum temperature',fontsize=11, fontweight='normal')
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax2.legend(loc='upper left',fontsize=8)
    
    # --------------------------------- MANIMUM ---------------------------
    ax1=plt.subplot(325)
    ax1.plot(RiparianTempB['times-annual_min'], RiparianTempB['temp-annual_min'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_min'], RiparianTempS30['temp-annual_min'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_min'], RiparianTempS60['temp-annual_min'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_min'], RiparianTempS90['temp-annual_min'],'m.-',label=r'S$_{90}$')

    ax1.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((256., 266))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax1.legend(loc='upper left',fontsize=8)
    ax1.text(2, 257., 'Annual minimum temperature',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(326)
    ax2.plot(HillslopeTempB['times-annual_min'], HillslopeTempB['temp-annual_min'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_min'], HillslopeTempS30['temp-annual_min'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_min'], HillslopeTempS60['temp-annual_min'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_min'], HillslopeTempS90['temp-annual_min'],'m.-',label=r'S$_{90}$')

    ax2.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    #ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((256., 266))
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax2.legend(loc='upper left',fontsize=8)
    ax2.text(2, 264.5, 'Annual minimum temperature',fontsize=11, fontweight='normal')
    ax2.legend(loc='upper right', fontsize=10, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    
    
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-%scm.pdf'%depth_cm, bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMinBAR():
    fig, axs = plt.subplots(3,2, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(321)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    ax1.set_title('Wetland',fontweight='normal')
    ax1.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(322)
    X = [HillslopeTempB['temp-annual_avg'][-1], HillslopeTempS30['temp-annual_avg'][-1],HillslopeTempS60['temp-annual_avg'][-1],HillslopeTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax2.set_ylim((266.5- TempC, 271.5- TempC))
    ax2.set_title('Upland',fontweight='normal')
    ax2.text(-0.2, 270.5- TempC, 'Annual average',fontsize=11, fontweight='normal')
    
    # --------------------------------- MAXIMUM ---------------------------
    ax1=plt.subplot(323)
    X = [RiparianTempB['temp-annual_max'][-1], RiparianTempS30['temp-annual_max'][-1],RiparianTempS60['temp-annual_max'][-1],RiparianTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    ax1.set_ylim((275.5- TempC, 276.5- TempC))
    ax1.text(-0.2, 276.4- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(324)
    X = [HillslopeTempB['temp-annual_max'][-1], HillslopeTempS30['temp-annual_max'][-1],HillslopeTempS60['temp-annual_max'][-1],HillslopeTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax2.set_ylim((275.5- TempC, 276.5- TempC))
    ax2.text(-0.2, 276.4- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    
    # --------------------------------- MANIMUM ---------------------------
    ax1=plt.subplot(325)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    #ax1.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    
    ax1.set_ylim((256.- TempC, 266- TempC))
    ax1.text(-0.2, 264.6- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(326)
    X = [HillslopeTempB['temp-annual_min'][-1], HillslopeTempS30['temp-annual_min'][-1],HillslopeTempS60['temp-annual_min'][-1],HillslopeTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    
    #ax2.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax2.set_ylim((256.- TempC, 266- TempC))
    ax2.text(-0.2, 264.5- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    ax2.legend(loc='upper right', fontsize=10, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-50cm-bar.pdf', bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [23]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMinBARSingleColumn():
    fig, axs = plt.subplots(3,1, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(311)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='black')
    
    X = [HillslopeTempB['temp-annual_avg'][-1], HillslopeTempS30['temp-annual_avg'][-1],HillslopeTempS60['temp-annual_avg'][-1],HillslopeTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos+0.25,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='cyan', hatch="/")
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    ax1.set_ylim((-6.0, 0))
    #ax1.set_title('Wetland',fontweight='normal')
    ax1.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()

    # --------------------------------- MAXIMUM ---------------------------
    ax2=plt.subplot(312)
    X = [RiparianTempB['temp-annual_max'][-1], RiparianTempS30['temp-annual_max'][-1],RiparianTempS60['temp-annual_max'][-1],RiparianTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='black')
    
    X = [HillslopeTempB['temp-annual_max'][-1], HillslopeTempS30['temp-annual_max'][-1],HillslopeTempS60['temp-annual_max'][-1],HillslopeTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos+0.25,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='cyan', hatch="/")
    
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax2.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    
    #ax2.set_ylim((275.5- TempC, 276.5- TempC))
    ax2.set_ylim((0, 8))
    #ax1.set_title('Wetland',fontweight='normal')
    ax2.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax3=plt.subplot(313)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='black')
    
    X = [HillslopeTempB['temp-annual_min'][-1], HillslopeTempS30['temp-annual_min'][-1],HillslopeTempS60['temp-annual_min'][-1],HillslopeTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos+0.25,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='cyan', hatch="/")
    
    ax3.set_xticks(xpos)
    ax3.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax3.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    
    #ax3.set_ylim((256.- TempC, 266- TempC))
    ax3.set_ylim((-18, 0))
    ax3.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()
    
    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [120]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMinBARSingleColumn1():
    fig, axs = plt.subplots(3,1, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(311)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    #ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    ax1.set_ylim((-3.0, 0))
    #ax1.set_title('Wetland',fontweight='normal')
    ax1.text(0.5, -2.85, 'Annual average (wetland)',fontsize=11, fontweight='normal')
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax3=plt.subplot(312)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos,X,align='center', alpha=1.0, color=['k', 'g','b','r'], width=0.25,edgecolor='black')
       
    ax3.set_xticks(xpos)
    ax3.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    
    ax3.set_ylim((-15, -10))
    ax3.text(0.5, -14.85, 'Annual minimum (wetland)',fontsize=11, fontweight='normal')
    
    # --------------------------------- Annual time series ---------------------------
    ax3=plt.subplot(313)
    x=np.linspace(0,364,365)
    ax3.plot(x,-273.15+WetlandTempB['temp'],'k-',label='Basecase')
    ax3.plot(x,-273.15+WetlandTempS30['temp'],'g-',label=r'S$_{30}$')
    ax3.plot(x,-273.15+WetlandTempS60['temp'],'b-',label=r'S$_{60}$')
    ax3.plot(x,-273.15+WetlandTempS90['temp'],'r-',label=r'S$_{90}$')

    ax3.set_xlim(75,290)
    #ax3.set_xticks(np.linspace(1,19.6,6))
    #ax3.set_xticklabels(np.linspace(19.2*,365,6),fontsize=11, fontweight='normal')
    
    ax3.set_ylabel(u'Soil temperature [\u00B0C]',fontsize=12, fontweight='normal')
    ax3.set_xlabel('Time [day of year]',fontsize=12, fontweight='normal')
    ax3.text(76., -4.9, 'Depth = 56 cm',fontsize=11, fontweight='normal')
    #ax3.set_ylim((-15, -10))
    ax3.legend(loc='lower right', fontsize=10,labelspacing=0.1,bbox_to_anchor=(0.9,-0.01, .1, .1))
    plt.tight_layout()
    
    
    plt.savefig(outfile+'avg-min-temp-385cm-bar.pdf', bbox_inches='tight', dpi=100)
    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [121]:
PlotAnnualAvgMaxMinBARSingleColumn1()

In [57]:
print WetlandTempB.keys()

['temp-annual_avg', 'temp-annual_min', 'temp-annual_max', 'times']


In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMinLine():
    fig, axs = plt.subplots(3,1, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(311)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    Y = [HillslopeTempB['temp-annual_avg'][-1], HillslopeTempS30['temp-annual_avg'][-1],HillslopeTempS60['temp-annual_avg'][-1],HillslopeTempS90['temp-annual_avg'][-1]]
    Y = [x - TempC for x in Y]
    ax1.plot(xpos,X,'r*-',label='Wetland')
    ax1.plot(xpos,Y,'k*-',label='Upland')
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel('Annual average [C]',fontsize='large', fontweight='normal')
    ax1.set_ylim((266.15 - TempC, 271.15- TempC))
    #ax1.set_title('Wetland',fontweight='normal')
    #ax1.text(3.2, 267- TempC, 'Annual average',fontsize=11, fontweight='normal')
    ax1.set_title('Soil temperatures at wetland and upland',fontsize=11, fontweight='normal')
    ax1.legend(loc='upper left', fontsize=10)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    plt.tight_layout()

    
    # --------------------------------- MAXIMUM ---------------------------
    ax1=plt.subplot(312)
    X = [RiparianTempB['temp-annual_max'][-1], RiparianTempS30['temp-annual_max'][-1],RiparianTempS60['temp-annual_max'][-1],RiparianTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    Y = [HillslopeTempB['temp-annual_max'][-1], HillslopeTempS30['temp-annual_max'][-1],HillslopeTempS60['temp-annual_max'][-1],HillslopeTempS90['temp-annual_max'][-1]]
    Y = [x - TempC for x in Y]
    ax1.plot(xpos,X,'r*-',label='Wetland')
    ax1.plot(xpos,Y,'k*-',label='Upland')
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    ax1.set_ylabel('Annual maximum [C]',fontsize='large', fontweight='normal')
    ax1.set_ylim((280.15- TempC, 281.15- TempC))
    #ax1.text(3.2, 275.6- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    
    ax1.legend(loc='upper left', fontsize=10)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax1=plt.subplot(313)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    Y = [HillslopeTempB['temp-annual_min'][-1], HillslopeTempS30['temp-annual_min'][-1],HillslopeTempS60['temp-annual_min'][-1],HillslopeTempS90['temp-annual_min'][-1]]
    Y = [x - TempC for x in Y]
    ax1.plot(xpos,X,'r*-',label='Wetland')
    ax1.plot(xpos,Y,'k*-',label='Upland')
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax1.set_xlabel('Simulation scenario',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Annual minimum [C]',fontsize='large', fontweight='normal')
    
    ax1.set_ylim((255.15- TempC, 259.15- TempC))
    #ax1.text(3.2, 264.6- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    ax1.legend(loc='upper left', fontsize=10)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    plt.tight_layout()
    
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-%scm-line.pdf'%depth_cm, bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
#PlotAnnualAverage()
#PlotAnnualAvgMaxMin()
#PlotAnnualAvgMaxMinBAR()
PlotAnnualAvgMaxMinLine()

In [ ]:
print 0.71*365, 0.8*365, .2*365, 0.4*365, 0.6*365
print 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 15#sept
print 31 + 28 + 31 + 30 + 31 # 146
print 31 + 28 + 31 + 30 + 31 + 30 + 31 #219